In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from langdetect import detect as detect_language

In [2]:
p = 'combined_agency_data_June_27.csv'

In [3]:
df = pd.read_csv(p)

# Remove empty textstring cells

In [4]:
cleaned_df = df[df.Textstring.notnull()]

# Removing html tags

In [5]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [6]:
cleaned_df.Textstring = cleaned_df.Textstring.apply(lambda x : cleanhtml(x))

/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [8]:
untagged_df = cleaned_df

# Detect language of the text

In [11]:
def detect_language_(s):
    try:
        language = detect_language(s)
        return language
    except:
        return ''

In [12]:
untagged_df['language'] = untagged_df.Textstring.apply(lambda x : detect_language_(x))

/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
augmented_df = untagged_df

# Remove unlabelled data points

In [15]:
labelled_df = augmented_df[augmented_df['Comment Ala'].notnull()]

# Find forbidden data points

In [17]:
for i in range(len(labelled_df)):
    content = labelled_df.Textstring.iloc[i]
    if 'forbidden' in content:
        labelled_df.loc[i, 'drop'] = True
    else:
        labelled_df.loc[i, 'drop'] = False

/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


KeyError: False

In [22]:
valid_points_df = labelled_df[labelled_df['drop']== False]

In [25]:
del valid_points_df['drop']

# Rephrase label

In [27]:
def rename_label(label):
    if isinstance(label, float) or label.lower() == 'no agency':
        return False
    else:
        return True

In [28]:
valid_points_df = valid_points_df.rename(columns={'Comment Ala': 'label'})

In [29]:
valid_points_df.label = valid_points_df.label.apply(lambda x : rename_label(x))

# Restrict to english language

In [30]:
df_en = valid_points_df[valid_points_df.language=='en']

In [31]:
df_en.to_csv('agency_data_en.csv')

In [32]:
df_en.label.value_counts()

False    897
True     720
Name: label, dtype: int64